# Predicting next word in a sentence 
## word here is represented by a character
Dataset is not published here, but for an idea, training is done on sentences that look like <b>AEDF234TH</b> and testing is done on sentences like <b>ASE452DG?</b>.

This is a standard Neural Network implementation of this problem, using [Keras](https://keras.io/).

More details on Data preprocessiong and Dataset preparation can be found here in the [LSTM implementation](AlienChat_LSTM_Keras.ipynb) of the problem


The best `train` accuracy of the model (with least `validation` loss,  achieved as the following : `

| Total Hidden layers      | Neurons / Hidden Layer | Epochs     | Train (loss / accuracy)| Validation (loss / accuracy)| Test (loss / accuracy)|
| :---        |    :----:   |    :----:   |    :----:   |    :----:   |    :----:   |
| 2      | 100       | 60   | 0.8924 / **69.08%** | 2.4394 / 38.70% | 0.9421 / **70.18%*|


In [90]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
import json

np.random.seed(1)

import tensorflow as tf
from keras.layers.core import Dense, Activation, Dropout
from keras.models import Sequential
from keras.layers import Dense, Flatten
from keras.optimizers import Adam
from alienchat import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [91]:
### Utils for this problem

SENTENCE_LENGTH = 9

ALPHABET = [chr(i) for i in range(65, 91)]
GRAMMAR_MAP = {}

for c in ALPHABET :
    GRAMMAR_MAP[c] = ord(c) - ord('A')

for i in range(1, 7+1) :
    GRAMMAR_MAP[str(i)] = GRAMMAR_MAP['Z'] + 1 + (ord(str(i)) - ord('1'))

REVERSE_GRAMMAR_MAP = ['']*len(GRAMMAR_MAP)

for char in GRAMMAR_MAP:
    REVERSE_GRAMMAR_MAP[GRAMMAR_MAP[char]] = str(char)
    
json.dumps(GRAMMAR_MAP)

'{"N": 13, "O": 14, "P": 15, "4": 29, "C": 2, "H": 7, "3": 28, "T": 19, "A": 0, "V": 21, "6": 31, "G": 6, "Q": 16, "K": 10, "I": 8, "1": 26, "R": 17, "S": 18, "Y": 24, "W": 22, "X": 23, "B": 1, "2": 27, "F": 5, "L": 11, "7": 32, "E": 4, "Z": 25, "M": 12, "J": 9, "D": 3, "5": 30, "U": 20}'

In [92]:
# Parameters

DECODED_PART_LENGTH = SENTENCE_LENGTH - 1
output_labels = len(GRAMMAR_MAP) # Number of output labels
CHUNK_LENGTH = 1

print("The input vector will have the shape {}x{}."
      .format(DECODED_PART_LENGTH//CHUNK_LENGTH, CHUNK_LENGTH*output_labels))

parent_dir = '.'
COLAB = False
if COLAB :
    from google.colab import drive
    drive.mount('/content/drive')
    parent_dir = '/content/drive/My Drive/LSTM'
    
train_file = parent_dir+'/'+'data/train.csv'
test_file = parent_dir+'/'+'data/answers.csv'



The input vector will have the shape 8x33.


In [93]:
from alienchat_dataset import DataSet
alienchat = DataSet(GRAMMAR_MAP, SENTENCE_LENGTH, validation_split=0,chunk_length=CHUNK_LENGTH)\
            .load_data(train_file, test_file)
print('Training data : ', alienchat.train.data.shape, 'Training labels : ', alienchat.train.labels.shape)
print('Validation data : ', alienchat.validation.data.shape, 'Validation labels : ', alienchat.validation.labels.shape)
print('Test data : ', alienchat.test.data.shape, 'Test labels : ', alienchat.test.labels.shape)


Training data :  (2543, 8, 33) Training labels :  (2543, 33)
Validation data :  (0,) Validation labels :  (0,)
Test data :  (379, 8, 33) Test labels :  (379, 33)


In [94]:
model = Sequential([
    Flatten(input_shape=(DECODED_PART_LENGTH//CHUNK_LENGTH, CHUNK_LENGTH*output_labels)),
    Dense(100, activation='relu'),
    Dropout(0.3),
    Dense(100, activation='relu'),
    Dropout(0.3),
    Dense(output_labels)
])

In [95]:
model.compile(optimizer=Adam(learning_rate=0.001),
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['acc'])

In [96]:
model.fit(alienchat.train.data, alienchat.train.labels, epochs=60, validation_split=.2)

Epoch 1/60
64/64 [==============================] - 0s 2ms/step - acc: 0.1323 - loss: 3.0961 - val_loss: 2.6210 - val_acc: 0.2122
Epoch 2/60
64/64 [==============================] - 0s 1ms/step - acc: 0.2119 - loss: 2.6147 - val_loss: 2.4683 - val_acc: 0.2554
Epoch 3/60
64/64 [==============================] - 0s 1ms/step - acc: 0.2822 - loss: 2.4273 - val_loss: 2.3344 - val_acc: 0.3281
Epoch 4/60
64/64 [==============================] - 0s 1ms/step - acc: 0.3402 - loss: 2.2946 - val_loss: 2.2488 - val_acc: 0.3399
Epoch 5/60
64/64 [==============================] - 0s 1ms/step - acc: 0.3599 - loss: 2.1821 - val_loss: 2.1714 - val_acc: 0.3674
Epoch 6/60
64/64 [==============================] - 0s 1ms/step - acc: 0.3805 - loss: 2.0787 - val_loss: 2.1426 - val_acc: 0.3752
Epoch 7/60
64/64 [==============================] - 0s 1ms/step - acc: 0.4090 - loss: 2.0013 - val_loss: 2.1168 - val_acc: 0.3851
Epoch 8/60
64/64 [==============================] - 0s 1ms/step - acc: 0.4248 - loss: 1.95

In [99]:
test_loss, test_acc = model.evaluate(alienchat.test.data, alienchat.test.labels, verbose=2)

12/12 - 0s - acc: 0.7230 - loss: 0.8941


In [100]:

def secondmax(arr) :
    b = np.sort(arr)
    return np.where(arr == b[-2])[0][0]


In [101]:
test = alienchat.test.rawdata
predictions = model.predict(alienchat.test.data)
test["predicted_word"] = [REVERSE_GRAMMAR_MAP[np.argmax(prediction)] for prediction in predictions]
test["second_best_pred"] = [REVERSE_GRAMMAR_MAP[secondmax(prediction)] for prediction in predictions]
#print(validate['predicted_word'])
test["encoded_word"] = test["sentence"].apply(lambda x : x[-1])
test[["sentence","predicted_word","encoded_word", "second_best_pred"]][test["encoded_word"] != test['predicted_word']].head(10)

,sentence,predicted_word,encoded_word,second_best_pred
2,2222222M2,M,2,S
10,R2SR2SR22,S,2,2
13,62MSRSSSR,E,R,R
14,2WKM1S2SR,S,R,G
18,SR22SR2SR,2,R,R
20,6YK2LLLZT,S,T,T
32,E22MMZEHP,5,P,P
35,SRSSGSRSR,2,R,R
38,6S2S2SEHP,5,P,P
42,2G2GSS1SR,S,R,G
